In [2]:
from utils import * 

import numpy as np
import pandas as pd
from pprint import pprint
import os
import matplotlib.pyplot as plt
from collections import defaultdict

# Gensim
from gensim.test.utils import datapath
from gensim.test.utils import common_texts, get_tmpfile

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.option_context('display.max_colwidth', 500);

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)


config = get_config('config.yaml')

[nltk_data] Error loading stopwords: <urlopen error [WinError 10060] A
[nltk_data]     connection attempt failed because the connected party
[nltk_data]     did not properly respond after a period of time, or
[nltk_data]     established connection failed because connected host
[nltk_data]     has failed to respond>
C:\Users\enlik\Miniconda3\lib\site-packages\sklearn\decomposition\_lda.py:28: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  EPS = np.finfo(np.float).eps


In [2]:
df = pd.read_csv(config['csv_input_local']['all_reviews_p3'], index_col=0)
df = df.reset_index(drop=True)
df

,userName,review,rating,date,appName
0,Hilary Meyer,Your drivers are great BUT your support is no ...,3,12/6/2020 19:38,bolt
1,Kenneth B,"Loving the app, but can only give it a medium ...",3,12/11/2020 14:06,bolt
2,Bonolo Mphahlele,The response time with regards to customer ser...,4,12/7/2020 13:59,bolt
3,jessfechi egolo,This has been happening for some time now and ...,1,12/13/2020 18:19,bolt
4,Safeeya Lawal,The drivers almost always never have change an...,3,12/5/2020 15:08,bolt
...,...,...,...,...,...
29808,P.Raimond,It’s rather difficult to get a taxi. Sometimes...,3,5/30/2018 5:37,yandexgo
29809,sõidan taksoga,Taxi drivers don’t speak Estonian or English. ...,1,11/11/2018 10:32,yandexgo
29810,madsp1r1t,Ride booking extension in Apple maps not avail...,3,11/7/2019 18:53,yandexgo
29811,hmerith,"I ordered a taxi, but when it arrived the driv...",1,7/12/2019 23:33,yandexgo


In [3]:
input_path = 'lda_mallet_model/mallet_lda_df_all_p3_v2.pkl'
df_topic_sents_keywords_m = pd.read_pickle(input_path)
df_topic_sents_keywords_m

,Dominant_Topic,Perc_Contribution,Keywords,0
0,18.0,0.1060,"card, cash, application, support, response, em...","[driver, support, computer, response, case, re..."
1,14.0,0.0948,"love, driver, destination, journey, cost, vehi...","[rating, moment, driver, rating, trip, driver,..."
2,11.0,0.0937,"customer, charge, company, scam, pay, refund, ...","[response, reimbursement, part, customer, prom..."
3,5.0,0.0737,"people, money, account, travel, amount, day, f...","[time, show, estimate, amount, destination, lo..."
4,18.0,0.0833,"card, cash, application, support, response, em...","[driver, option, change, time, transfer, trust..."
...,...,...,...,...
29808,7.0,0.0917,"taxi, location, map, app, point, airport, feat...","[difficult_get, taxi, time, takes_lot, time, t..."
29809,5.0,0.0833,"people, money, account, travel, amount, day, f...","[money, card, money, taxi]"
29810,19.0,0.0735,"app, option, user, star, notification, screen,...","[ride, extension, apple, map, apple_pay]"
29811,14.0,0.0834,"love, driver, destination, journey, cost, vehi...","[driver, destination, destination, mad]"


In [12]:
df_dominant_topic_m = find_dominant_topic_in_each_doc(df_topic_sents_keywords_m, df)
# print("Finding the dominant topic in each document")
df_dominant_topic_m.head(40).style.set_properties(subset=['review'], **{'width': '400px'})

,Dominant_Topic,Perc_Contribution,Keywords,Text,rating,review,appName
0,18.000000,0.106000,"card, cash, application, support, response, email, message, complaint, code, month","['driver', 'support', 'computer', 'response', 'case', 'response', 'support', 'discount', 'account', 'thought', 'client']",3,"Your drivers are great BUT your support is no good. Why can't it be like Uber where you can actually speak to someone. Instead of getting computer generated responses or like in my case, no response at all. The three stars are for the support. Also I have received a single discount to this account since I installed it three months ago. I'm very disappointed in Bolt, I thought they were there for their clients.",bolt
1,14.000000,0.094800,"love, driver, destination, journey, cost, vehicle, rating, pricing, bit, part","['rating', 'moment', 'driver', 'rating', 'trip', 'driver', 'apartment', 'ride', 'drive', 'destination']",3,"Loving the app, but can only give it a medium rating at the moment as the drivers you give a bad rating on still gets your trip. I had a bad experience with a driver outside my apartment, but he always gets my rides as he's parked there when with my one star and bad review (refused to drive to the destination, rude and aggressive).",bolt
2,11.000000,0.093700,"customer, charge, company, scam, pay, refund, estimate, care, fault, mistake","['response', 'reimbursement', 'part', 'customer', 'promo', 'part', 'situation', 'part', 'company', 'extra_money', 'people', 'card', 'dodgy', 'stick', 'amount', 'card']",4,The response time with regards to customer service was great and thank you. Kindly improve on the reimbursement part because it's not every customer who'll appreciate a promo. That part put some of us in a very sticky situation financially especially now that we have COVID-19. The last part would be for your company not to always take extra money just because people are paying with cards. That's dodgy. Stick to the same amount whether I'm paying cash or with a card.,bolt
3,5.000000,0.073700,"people, money, account, travel, amount, day, feedback, traveller, process, fact","['time', 'show', 'estimate', 'amount', 'destination', 'location', 'price', 'feedback', 'uninstall', 'rubbish', 'app']",1,This has been happening for some time now and i always keep quiet. You will show me an estimate of the amount for the destination and when i get to my location the price i will see will be way higher than what you showed. I will stop using you. Just decided to give this feedback before i uninstall this rubbish app,bolt
4,18.000000,0.083300,"card, cash, application, support, response, email, message, complaint, code, month","['driver', 'option', 'change', 'time', 'transfer', 'trust', 'card', 'cash', 'please_fix', 'problem', 'way']",3,"The drivers almost always never have change and I'm left with no option but to leave my 100-300 naira change with them which I obviously don't intend to. It's not all the time we're able to transfer and not everyone trust using their cards with the app, for those of us that use cash please fix this problem whatever way, it's really annoying.",bolt
5,11.000000,0.063900,"customer, charge, company, scam, pay, refund, estimate, care, fault, mistake","['driver', 'driver', 'thing', 'customer', 'service', 'stopped_using']",1,I requested a ride and the driver (Akhona) completed the trip without picking me up. Money was deducted from my bank account and I have reported this 2 days ago and nothing has been resolved. The driver is driving around freely and probably doing the same thing to other customers. I've been using Bolt for more than 4 years now and must say I'm shocked at the customer support service. I've since stopped using bolt cause I'm afraid the same thing will happen again. Very very disappointed.,bolt
6,18.000000,0.073500,"card, cash, application, support, response, email, message, complaint, code, month","['month', 'unknown_error', 'message', 'time', 'driver', 'cancel', 'place', 'lift', 'fix'

In [13]:
df_dominant_topic_m

,Dominant_Topic,Perc_Contribution,Keywords,Text,rating,review,appName
0,18.0,0.1060,"card, cash, application, support, response, em...","[driver, support, computer, response, case, re...",3,Your drivers are great BUT your support is no ...,bolt
1,14.0,0.0948,"love, driver, destination, journey, cost, vehi...","[rating, moment, driver, rating, trip, driver,...",3,"Loving the app, but can only give it a medium ...",bolt
2,11.0,0.0937,"customer, charge, company, scam, pay, refund, ...","[response, reimbursement, part, customer, prom...",4,The response time with regards to customer ser...,bolt
3,5.0,0.0737,"people, money, account, travel, amount, day, f...","[time, show, estimate, amount, destination, lo...",1,This has been happening for some time now and ...,bolt
4,18.0,0.0833,"card, cash, application, support, response, em...","[driver, option, change, time, transfer, trust...",3,The drivers almost always never have change an...,bolt
...,...,...,...,...,...,...,...
29808,7.0,0.0917,"taxi, location, map, app, point, airport, feat...","[difficult_get, taxi, time, takes_lot, time, t...",3,It’s rather difficult to get a taxi. Sometimes...,yandexgo
29809,5.0,0.0833,"people, money, account, travel, amount, day, f...","[money, card, money, taxi]",1,Taxi drivers don’t speak Estonian or English. ...,yandexgo
29810,19.0,0.0735,"app, option, user, star, notification, screen,...","[ride, extension, apple, map, apple_pay]",3,Ride booking extension in Apple maps not avail...,yandexgo
29811,14.0,0.0834,"love, driver, destination, journey, cost, vehi...","[driver, destination, destination, mad]",1,"I ordered a taxi, but when it arrived the driv...",yandexgo


In [14]:
df_dominant_topic_m.to_csv('all_reviews_p3_dominant_topic_m.csv')

In [8]:
print("Find the most representative document for each topic")
sent_topics_sorteddf_m = find_most_representative_doc_for_each_doc(df_topic_sents_keywords_m, df)
sent_topics_sorteddf_m.style.set_properties(subset=['review'], **{'width': '400px'})

Find the most representative document for each topic


,Dominant_Topic,Perc_Contribution,Topic_Keywords,star_rating,review
0,0.000000,0.149500,"time, pick, drivers_alway, experience, driver_cancel, occasion, comfort, spot, matter, mind",1,"In a number of occasions recently a driver accepts my trip, but s/he is completing a trip nearby. The time for the driver to arrive goes from say 5 mins to 9 mins - and after a few minutes the driver cancels! So by the time another driver arrives I have lost at least 15-20 minutes. Is this a new thing or have I been unlucky? I am irritated by the time it supposedly takes for the driver to get to me, then a cancellation which makes the whole experience stressful and wastes time."
1,1.000000,0.148400,"app, update, detail, every_time, country, pick_location, info, improvement, recommend, interface",3,"The GPS before the update was much more accurate. After the update, GPS points to a location so far away, I may as well walk to my destination. Also, forget about setting your location manually after the update... It simply places the pin to the closest road. Funny enough, used this app so often successfully on the same locations in the past. Since the update, this app has caused me issues, like waiting in the rain yesterday for over an hour to get 3 taxis, all of which, one after the next, ended up getting lost thanks to the GPS issues with the update. I wanted to be fair, and rated the app 3/5... the previous version would have been a 4, but the current version is more like a 2."
2,2.000000,0.173300,"trip, driver, payment, today, pay, case, client, transport, cancel_trip, difference",1,Total horrible ur rule From friday i m saying that give my extra money back 194Rs which payment already has been done previous trip then why friday trip again taken 194Rs extra i don't understand why i m not getting extra money back total payment i have to do 372Rs payment on friday i told that this payment already done then also uber driver saying that money uber company wil give u then y not til now i didn't get back extra money 194Rs y.
3,3.000000,0.236500,"driver, minute, request, traffic, job, pickup, morning, street, arrival, bus",3,"So we’ve been using Uber a couple of times a day whilst on holiday in the US. Literally every single time the estimated car arrival time is out by a great deal. Just now when confirming a trip, we did so based on the 4 minute away estimation of the nearest car. We waited outside our hotel for more like 14 minutes before the car actually got there. First the time went up to 5 minutes then to 7 minutes, then it just stayed 6 minutes for a few minutes and slowly started going back down again (not in a real-time minute counter but Uber’s much slower minute counter). There wasn’t really much traffic on the street so I don’t understand how can the estimation be out by this much. I would much prefer to know the actual estimated arrival time as opposed to standing by the road like an idiot. I definitely would have dried my hair this morning if I knew about the extra 10 minutes wait time, instead I was standing on the windy roadside for quarter of an hour with my wet hair. And this was pretty much the same with each ride ordered I’ve the past 10 days. Anywhere between double to triple the amount of estimated car arrival time we had to wait, not cool. Please make your app more accurate. For now I’m going to switch to Lyft for the last 4 days if our trip and see if they do a better estimation job. It’s just a shame as otherwise Uber would be a service I’d use and recommend more..."
4,4.000000,0.170200,"ride, day, show, search, end, destination, page, time, garbage, pass",1,The worst app U can use to con Urself......they say there ride cost is fluctuating......so a ride estimated 32/- was charged 62/- at the end of ride(same ride with UBER cost 27/-).......the ride estimated from jaipur railway station to jawahar circle is 100-110/- and end of journey it cost 135/- (same ride with uber cost 73/-).......there where many rides there after which l

In [ ]:
# df_dominant_topic_m['Num_Documents'].sum()

In [ ]:
# sent_topics_sorteddf_m.to_csv('test.csv')

## Average Rating per Topic

In [5]:
# df_dominant_topic_m
df_dominant_topic_m = pd.read_csv(config['csv_input_local']['all_reviews_p3_topic'], index_col=0)
df_dominant_topic_m

,Dominant_Topic,Perc_Contribution,Keywords,Text,rating,review,appName
0,18.0,0.1060,"card, cash, application, support, response, em...","['driver', 'support', 'computer', 'response', ...",3,Your drivers are great BUT your support is no ...,bolt
1,14.0,0.0948,"love, driver, destination, journey, cost, vehi...","['rating', 'moment', 'driver', 'rating', 'trip...",3,"Loving the app, but can only give it a medium ...",bolt
2,11.0,0.0937,"customer, charge, company, scam, pay, refund, ...","['response', 'reimbursement', 'part', 'custome...",4,The response time with regards to customer ser...,bolt
3,5.0,0.0737,"people, money, account, travel, amount, day, f...","['time', 'show', 'estimate', 'amount', 'destin...",1,This has been happening for some time now and ...,bolt
4,18.0,0.0833,"card, cash, application, support, response, em...","['driver', 'option', 'change', 'time', 'transf...",3,The drivers almost always never have change an...,bolt
...,...,...,...,...,...,...,...
29808,7.0,0.0917,"taxi, location, map, app, point, airport, feat...","['difficult_get', 'taxi', 'time', 'takes_lot',...",3,It’s rather difficult to get a taxi. Sometimes...,yandexgo
29809,5.0,0.0833,"people, money, account, travel, amount, day, f...","['money', 'card', 'money', 'taxi']",1,Taxi drivers don’t speak Estonian or English. ...,yandexgo
29810,19.0,0.0735,"app, option, user, star, notification, screen,...","['ride', 'extension', 'apple', 'map', 'apple_p...",3,Ride booking extension in Apple maps not avail...,yandexgo
29811,14.0,0.0834,"love, driver, destination, journey, cost, vehi...","['driver', 'destination', 'destination', 'mad']",1,"I ordered a taxi, but when it arrived the driv...",yandexgo


In [6]:
# get list of unique values sorted - https://www.geeksforgeeks.org/python-get-unique-values-list/
list_unique_topics = np.unique(df_dominant_topic_m['Dominant_Topic'])

In [7]:
for topic in list_unique_topics:
    avg_rating = round(df_dominant_topic_m[df_dominant_topic_m['Dominant_Topic'] == topic]['rating'].mean(), 2)
    print('Average Rating for Topic ' + str(topic) + ' = ' + str(avg_rating))

Average Rating for Topic 0.0 = 4.33
Average Rating for Topic 1.0 = 3.34
Average Rating for Topic 2.0 = 3.01
Average Rating for Topic 3.0 = 2.84
Average Rating for Topic 4.0 = 3.33
Average Rating for Topic 5.0 = 3.25
Average Rating for Topic 6.0 = 2.75
Average Rating for Topic 7.0 = 2.6
Average Rating for Topic 8.0 = 3.35
Average Rating for Topic 9.0 = 4.03
Average Rating for Topic 10.0 = 3.2
Average Rating for Topic 11.0 = 2.24
Average Rating for Topic 12.0 = 3.26
Average Rating for Topic 13.0 = 3.67
Average Rating for Topic 14.0 = 4.21
Average Rating for Topic 15.0 = 2.24
Average Rating for Topic 16.0 = 3.74
Average Rating for Topic 17.0 = 3.63
Average Rating for Topic 18.0 = 2.36
Average Rating for Topic 19.0 = 3.01


## Average Sentiment per Topic

In [8]:
df_dominant_topic_m_sentiment = pd.read_csv(config['csv_input_local']['all_reviews_p3_topic_sentiment'], index_col=0)
df_dominant_topic_m_sentiment = df_dominant_topic_m_sentiment.reset_index(drop=True)
df_dominant_topic_m_sentiment

,Dominant_Topic,Perc_Contribution,Keywords,Text,rating,review,appName,sentiment_score
0,18.0,0.1060,"card, cash, application, support, response, em...","['driver', 'support', 'computer', 'response', ...",3,Your drivers are great BUT your support is no ...,bolt,0
1,14.0,0.0948,"love, driver, destination, journey, cost, vehi...","['rating', 'moment', 'driver', 'rating', 'trip...",3,"Loving the app, but can only give it a medium ...",bolt,1
2,11.0,0.0937,"customer, charge, company, scam, pay, refund, ...","['response', 'reimbursement', 'part', 'custome...",4,The response time with regards to customer ser...,bolt,2
3,5.0,0.0737,"people, money, account, travel, amount, day, f...","['time', 'show', 'estimate', 'amount', 'destin...",1,This has been happening for some time now and ...,bolt,-2
4,18.0,0.0833,"card, cash, application, support, response, em...","['driver', 'option', 'change', 'time', 'transf...",3,The drivers almost always never have change an...,bolt,0
...,...,...,...,...,...,...,...,...
29808,7.0,0.0917,"taxi, location, map, app, point, airport, feat...","['difficult_get', 'taxi', 'time', 'takes_lot',...",3,It’s rather difficult to get a taxi. Sometimes...,yandexgo,0
29809,5.0,0.0833,"people, money, account, travel, amount, day, f...","['money', 'card', 'money', 'taxi']",1,Taxi drivers don’t speak Estonian or English. ...,yandexgo,-2
29810,19.0,0.0735,"app, option, user, star, notification, screen,...","['ride', 'extension', 'apple', 'map', 'apple_p...",3,Ride booking extension in Apple maps not avail...,yandexgo,0
29811,14.0,0.0834,"love, driver, destination, journey, cost, vehi...","['driver', 'destination', 'destination', 'mad']",1,"I ordered a taxi, but when it arrived the driv...",yandexgo,-2


In [9]:
for topic in list_unique_topics:
    avg_sentiment = round(df_dominant_topic_m_sentiment[df_dominant_topic_m_sentiment['Dominant_Topic'] == topic]['sentiment_score'].mean(),2)
    print('Average Sentiment for Topic ' + str(topic) + ' = ' + str(avg_sentiment))

Average Sentiment for Topic 0.0 = 1.5
Average Sentiment for Topic 1.0 = 0.48
Average Sentiment for Topic 2.0 = -0.01
Average Sentiment for Topic 3.0 = -0.22
Average Sentiment for Topic 4.0 = 0.38
Average Sentiment for Topic 5.0 = 0.3
Average Sentiment for Topic 6.0 = -0.26
Average Sentiment for Topic 7.0 = -0.49
Average Sentiment for Topic 8.0 = 0.37
Average Sentiment for Topic 9.0 = 1.18
Average Sentiment for Topic 10.0 = 0.22
Average Sentiment for Topic 11.0 = -0.98
Average Sentiment for Topic 12.0 = 0.3
Average Sentiment for Topic 13.0 = 0.8
Average Sentiment for Topic 14.0 = 1.4
Average Sentiment for Topic 15.0 = -0.86
Average Sentiment for Topic 16.0 = 0.84
Average Sentiment for Topic 17.0 = 0.72
Average Sentiment for Topic 18.0 = -0.74
Average Sentiment for Topic 19.0 = 0.05


In [ ]:
df_dominant_topic_m_sentiment[df_dominant_topic_m_sentiment['Dominant_Topic'] == 1.0]['sentiment_score'].sum()

In [14]:
# 394 / len(df_dominant_topic_m_sentiment[df_dominant_topic_m_sentiment['Dominant_Topic'] == 1.0])

## Further Analysis

In [13]:
df_dominant_topic_m[df_dominant_topic_m['Dominant_Topic'] == 0.0].head(20000)

,Dominant_Topic,Perc_Contribution,Keywords,Text,rating,review,appName
37,0.0,0.0500,"time, pick, drivers_alway, experience, driver_...",[],5,"Wonderful drive. Once you place your order, in...",bolt
65,0.0,0.0655,"time, pick, drivers_alway, experience, driver_...","['increase_price', 'time']",1,It is not fair to increase prices when i have ...,bolt
92,0.0,0.0882,"time, pick, drivers_alway, experience, driver_...","['app', 'driver', 'missleading', 'time', 'pick...",1,Horrible app. The driver movement is missleadi...,bolt
104,0.0,0.0752,"time, pick, drivers_alway, experience, driver_...","['app', 'ride', 'waiting_time', 'time', 'pop',...",3,"In general the app is ok, but I do not like wh...",bolt
111,0.0,0.0500,"time, pick, drivers_alway, experience, driver_...",[],5,I've never had issues with the app or the driv...,bolt
...,...,...,...,...,...,...,...
29745,0.0,0.0668,"time, pick, drivers_alway, experience, driver_...",['time'],5,Nice alternative to Uber in Moscow and St Pete...,yandexgo
29754,0.0,0.0588,"time, pick, drivers_alway, experience, driver_...","['set', 'icon', 'app', 'choice']",1,"Awful, set's me up third time! I want to see t...",yandexgo
29768,0.0,0.0805,"time, pick, drivers_alway, experience, driver_...","['program', 'send_confirmation', 'sms', 'time'...",1,"I tried the program twice, with a lag of two w...",yandexgo
29777,0.0,0.0660,"time, pick, drivers_alway, experience, driver_...","['mind', 'use', 'whatsapp', 'yandex']",1,"Terrible, when I'm on a different app, the yan...",yandexgo


## Topic Distribution Per App

In [3]:
df_dominant_topic_m_sentiment = pd.read_csv(config['csv_input_local']['all_reviews_p3_topic_sentiment'], index_col=0)
df_dominant_topic_m_sentiment

,Dominant_Topic,Perc_Contribution,Keywords,Text,rating,review,appName,sentiment_score
0,18.0,0.1060,"card, cash, application, support, response, em...","['driver', 'support', 'computer', 'response', ...",3,Your drivers are great BUT your support is no ...,bolt,0
1,14.0,0.0948,"love, driver, destination, journey, cost, vehi...","['rating', 'moment', 'driver', 'rating', 'trip...",3,"Loving the app, but can only give it a medium ...",bolt,1
2,11.0,0.0937,"customer, charge, company, scam, pay, refund, ...","['response', 'reimbursement', 'part', 'custome...",4,The response time with regards to customer ser...,bolt,2
3,5.0,0.0737,"people, money, account, travel, amount, day, f...","['time', 'show', 'estimate', 'amount', 'destin...",1,This has been happening for some time now and ...,bolt,-2
4,18.0,0.0833,"card, cash, application, support, response, em...","['driver', 'option', 'change', 'time', 'transf...",3,The drivers almost always never have change an...,bolt,0
...,...,...,...,...,...,...,...,...
29808,7.0,0.0917,"taxi, location, map, app, point, airport, feat...","['difficult_get', 'taxi', 'time', 'takes_lot',...",3,It’s rather difficult to get a taxi. Sometimes...,yandexgo,0
29809,5.0,0.0833,"people, money, account, travel, amount, day, f...","['money', 'card', 'money', 'taxi']",1,Taxi drivers don’t speak Estonian or English. ...,yandexgo,-2
29810,19.0,0.0735,"app, option, user, star, notification, screen,...","['ride', 'extension', 'apple', 'map', 'apple_p...",3,Ride booking extension in Apple maps not avail...,yandexgo,0
29811,14.0,0.0834,"love, driver, destination, journey, cost, vehi...","['driver', 'destination', 'destination', 'mad']",1,"I ordered a taxi, but when it arrived the driv...",yandexgo,-2


### Mapping

In [4]:
topic_map = {6.0 : "T1",
            9.0 : "T1",
            8.0 : "T2",
            15.0 : "T2",
            5.0 : "T3",
            12.0 : "T3",
            14.0 : "T3",
            17.0 : "T4",
            3.0 : "T5",
            9.0 : "T6",
            0.0 : "T7",
            1.0 : "T7",
            11.0 : "T8",
            16.0 : "T9",
            19.0 : "T9",
            7.0 : "T10",
            4.0 : "T11",
            15.0 : "T11",
            13.0 : "T11",
            10.0 : "T12",
            2.0 : "T13",
            18.0 : "T13",
            }

In [5]:
df_dominant_topic_m_sentiment['final_topic'] = df_dominant_topic_m_sentiment['Dominant_Topic'].map(topic_map)
# df_dominant_topic_m_sentiment[df_dominant_topic_m_sentiment['final_topic'] == "20"]
df_dominant_topic_m_sentiment

,Dominant_Topic,Perc_Contribution,Keywords,Text,rating,review,appName,sentiment_score,final_topic
0,18.0,0.1060,"card, cash, application, support, response, em...","['driver', 'support', 'computer', 'response', ...",3,Your drivers are great BUT your support is no ...,bolt,0,T13
1,14.0,0.0948,"love, driver, destination, journey, cost, vehi...","['rating', 'moment', 'driver', 'rating', 'trip...",3,"Loving the app, but can only give it a medium ...",bolt,1,T3
2,11.0,0.0937,"customer, charge, company, scam, pay, refund, ...","['response', 'reimbursement', 'part', 'custome...",4,The response time with regards to customer ser...,bolt,2,T8
3,5.0,0.0737,"people, money, account, travel, amount, day, f...","['time', 'show', 'estimate', 'amount', 'destin...",1,This has been happening for some time now and ...,bolt,-2,T3
4,18.0,0.0833,"card, cash, application, support, response, em...","['driver', 'option', 'change', 'time', 'transf...",3,The drivers almost always never have change an...,bolt,0,T13
...,...,...,...,...,...,...,...,...,...
29808,7.0,0.0917,"taxi, location, map, app, point, airport, feat...","['difficult_get', 'taxi', 'time', 'takes_lot',...",3,It’s rather difficult to get a taxi. Sometimes...,yandexgo,0,T10
29809,5.0,0.0833,"people, money, account, travel, amount, day, f...","['money', 'card', 'money', 'taxi']",1,Taxi drivers don’t speak Estonian or English. ...,yandexgo,-2,T3
29810,19.0,0.0735,"app, option, user, star, notification, screen,...","['ride', 'extension', 'apple', 'map', 'apple_p...",3,Ride booking extension in Apple maps not avail...,yandexgo,0,T9
29811,14.0,0.0834,"love, driver, destination, journey, cost, vehi...","['driver', 'destination', 'destination', 'mad']",1,"I ordered a taxi, but when it arrived the driv...",yandexgo,-2,T3


In [6]:
# df = df_dominant_topic_m_sentiment[['appName','final_topic','review']].groupby(['appName','Dominant_Topic']).count().reset_index()
df_dist = df_dominant_topic_m_sentiment[['appName','final_topic','review']].groupby(['appName','final_topic']).count()
df_dist['percentage'] = df_dist.groupby(level=0).apply(lambda x: 100 * x / float(x.sum()))
df_dist

review  percentage
appName   final_topic                    
blablacar T1              103    3.286535
          T10              74    2.361200
          T11             412   13.146139
          T12              72    2.297384
          T13             250    7.977026
          T2              142    4.530951
          T3              680   21.697511
          T4              137    4.371410
          T5               46    1.467773
          T6              132    4.211870
          T7              664   21.186981
          T8               58    1.850670
          T9              364   11.614550
bolt      T1              422    3.911391
          T10             404    3.744555
          T11            1073    9.945315
          T12             377    3.494300
          T13             957    8.870146
          T2              389    3.605524
          T3             1355   12.559088
          T4              697    6.460284
          T5              434    4.022616
          T6              602    5.579757
          T7             2568   23.802021
          T8              328    3.040133
          T9             1183   10.964872
cabify    T1               21    3.839122
          T10              39    7.129799
          T11              57   10.420475
          T12              41    7.495430
          T13              61   11.151737
          T2               38    6.946984
          T3               52    9.506399
          T4               15    2.742230
          T5               19    3.473492
          T6               47    8.592322
          T7               87   15.904936
          T8               15    2.742230
          T9               55   10.054845
freenow   T1               92    3.389831
          T10             421   15.512159
          T11             195    7.184967
          T12             125    4.605748
          T13             229    8.437730
          T2               74    2.726603
          T3              201    7.406043
          T4              136    5.011054
          T5              111    4.089904
          T6              147    5.416360
          T7              636   23.434046
          T8               85    3.131909
          T9              262    9.653648
getaround T1               18    7.317073
          T10               3    1.219512
          T11               9    3.658537
          T12               8    3.252033
          T13              15    6.097561
          T2               61   24.796748
          T3               36   14.634146
          T4                9    3.658537
          T5                7    2.845528
          T6               13    5.284553
          T7               36   14.634146
          T8               20    8.130081
          T9               11    4.471545
olacabs   T1              204    5.723906
          T10             203    5.695847
          T11             934   26.206510
          T12             128    3.591470
          T13             368   10.325477
          T2              100    2.805836
          T3              430   12.065095
          T4              103    2.890011
          T5              115    3.226712
          T6              109    3.058361
          T7              287    8.052750
          T8              223    6.257015
          T9              360   10.101010
taxieu    T10               9   20.930233
          T11               1    2.325581
          T12               1    2.325581
          T13               4    9.302326
          T2                1    2.325581
          T3                1    2.325581
          T4                4    9.302326
          T5                1    2.325581
          T6                2    4.651163
          T7               11   25.581395
          T8                1    2.325581
          T9                7   16.279070
uber      T1              293    4.342027
          T10             287    4.253112
          T11             804   11.914641
          T12             263    

## Average Rating and Sentiment per App